# QKD

### Key Generation
This step serves as the implementation of the generation of the key.

In [25]:
import cirq
from random import choices

In [35]:
def gen_bin_key(min_key_length: int) -> str:
    """
    Generate a random key (0's and 1's) not less than a specified length.

    Parameters:
    - min_key_length (int): The minimum length of the generated key.

    Returns:
    - str: The generated key if no eavesdropper is detected, otherwise "There was an eavesdropper!".

    Algorithm:
    1. First Person key Generation:
        - Generate random bits for the encoder key and encoder bases.
        - Create an encoder circuit using Cirq.
        - Apply the appropriate gates based on the encoder key and encoder bases to each qubit.
    2. Second Person key Reception:
        - Generate random bases for the decoder.
        - Create a decoder circuit using Cirq.
        - Apply the appropriate gates based on the decoder bases to each qubit.
        - Measure the qubits and store the measurement results as the second person key.
    3. Compare:
        - Combine the encoder and decoder circuits into a single BB84 circuit.
        - Simulate the circuit using Cirq.
        - Retrieve the measurement results for the second person key.
        - Compare the encoder and decoder keys based on the matching bases.
    4. Convert half of the key:
        - Determine the number of bits to compare based on the length of the encoder key.
        - If the compared portions of the encoder and decoder keys match, discard the compared portion.
        - Return the remaining portion of the encoder key as the final key if no eavesdropper is detected.
        - Otherwise, return "There was an eavesdropper!".
    """

    # First Person key Generation
    encode_gates = {0: cirq.I, 1: cirq.X}
    basis_gates = {"X": cirq.H, "Z": cirq.I}

    num_bits = min_key_length * 4 # Excess bits to account for those we use to compare
    encoder_key = choices([0, 1], k=num_bits)
    encoder_bases = choices(["Z", "X"], k=num_bits)

    encoder_circuit = cirq.Circuit()
    qubits = cirq.NamedQubit.range(num_bits, prefix="q")

    for bit in range(num_bits):
        encode_value = encoder_key[bit]
        encode_gate = encode_gates[encode_value]

        basis_value = encoder_bases[bit]
        basis_gate = basis_gates[basis_value]

        qubit = qubits[bit]
        encoder_circuit.append(encode_gate(qubit))
        encoder_circuit.append(basis_gate(qubit))

    # Second Person key Reception
    decoder_bases = choices(["Z", "X"], k=num_bits)

    decoder_circuit = cirq.Circuit()
    for bit in range(num_bits):
        basis_value = decoder_bases[bit]
        basis_gate = basis_gates[basis_value]

        qubit = qubits[bit]
        decoder_circuit.append(basis_gate(qubit))

    decoder_circuit.append(cirq.measure(qubits, key="second person key"))

    # Compare
    bb84_circuit = encoder_circuit + decoder_circuit

    sim = cirq.Simulator()
    results = sim.run(bb84_circuit)
    decoder_key = results.measurements["second person key"][0]

    final_encoder_key = []
    final_decoder_key = []
    for bit in range(num_bits):
        if encoder_bases[bit] == decoder_bases[bit]:
            final_encoder_key.append(encoder_key[bit])
            final_decoder_key.append(decoder_key[bit])

    # Convert halve of the key
    num_bits_to_compare = int(len(final_encoder_key) * .5)
    if final_encoder_key[0:num_bits_to_compare] == final_decoder_key[0:num_bits_to_compare]:
        final_encoder_key = final_encoder_key[num_bits_to_compare:]
        final_decoder_key = final_decoder_key[num_bits_to_compare:]

        return "".join(str(bit) for bit in final_encoder_key)

    return "There was an eavesdropper!"


In [23]:
def unicode_to_binary(text: str) -> str:
    """
    Convert a unicode string to binary.
    """
    return ''.join(format(ord(char), '08b') for char in text)


In [6]:
def binary_to_unicode(binary: str) -> str:
    """
    Convert a binary string to a unicode text
    """
    return ''.join([chr(int(binary[i:i+8], 2)) for i in range(0, len(binary), 8)])

In [80]:
def encrypt(text: str, key: str) -> str:
    """
    Encrypt a text using a key.

    Parameters:
    - text (str): The text to encrypt.
    - key (str): The key to use for encryption. Should be binary.

    Returns:
    - str: The encrypted text.
    """
    text = unicode_to_binary(text)
    while len(key) < len(text):
        key += key

    encrypted_text = ""
    for i in range(len(text)):
        encrypted_text += str(int(text[i]) ^ int(key[i]))

    return binary_to_unicode(encrypted_text)

In [89]:
def decrypt(text: str, key: str) -> str:
    """
    Decrypt a text using a key.

    Parameters:
    - text (str): The text to decrypt.
    - key (str): The key to use for decryption. Should be binary.

    Returns:
    - str: The decrypted text.
    """
    binary_encrypted_text = unicode_to_binary(text)
    while len(key) < len(binary_encrypted_text):
        key += key
    
    decrypted_text = ""
    for i in range(len(binary_encrypted_text)):
        decrypted_text += str(int(binary_encrypted_text[i]) ^ int(key[i]))
    
    return binary_to_unicode(decrypted_text)

### Example Usage

In [94]:
msg = "Hello, World!"
key_strength = 100

print("Message:", msg)
key = gen_bin_key(key_strength)
print("Key:", key)
encrypted_text = encrypt(msg, key)
print("Encrypted text:", encrypted_text)
decrypted_text = decrypt(encrypted_text, key)
print("Decrypted text:", decrypted_text)

Message: Hello, World!
Key: 100101010001000101000011001110010111011010101111101110110011000011011100100101101000001101010010001101
Encrypted text: Ýt/Ug³äï6
Decrypted text: Hello, World!
